In [1]:
import numpy as np 
import pandas as pd 
from binascii import hexlify        #Used to convert Ascii to hex

In [2]:
def print_poly(arr):
    '''
    Returns polynomial in string format to be printed
    '''
    disp = '( ' + str(arr[0])
    for i in range(1,len(arr)):
        if (arr[i]==1):
            disp = disp + " + x^" + str(i)
    disp = disp + " ) "
    return disp

In [3]:
def convert_bin_array_to_hex(arr):
    '''
    Converts an array representaion of a binary number back to it's hexadecimal code
    Argument: arr: binary number array
    Return: code: hexadecimal code
    '''

    val = 0
    for i in range(7,-1,-1):
        val = val*2 + int(arr[i])
    code = hex(val)
    return code

In [4]:
def convert_hex_to_bin_array(code):
    '''
    Converts an 8bit hexadecimal number into it's binary representation in array form
    Argument: code -> hexadecimal code
    Return type: arr -> numpy array of size 9 containing code's binary representation
    '''
    val = int(code,16)
    arr = np.zeros(9,dtype = int)

    for i in range(0,8):
        arr[i] = int(val%2)
        val = val//2

    return arr

In [5]:
def field_div(dividend,divisor, quotient=None):
    '''
    Perform Field Division in Polynomial Ring Z_2[x]
    Arguments: List-> Divisor, Dividend, quotient
                quotient ->Needed for individual steps of long division
    Return: Quotient, Remainder lists
    '''
    
    deg_dividend = 0
    length = len(dividend)

    for i in range (0,len(dividend)):
        if dividend[i]!=0 :
            deg_dividend = i 
    deg_divisor = 0
    for i in range (0,len(divisor)):
        if divisor[i]!=0 :
            deg_divisor = i 

    #Initializing quotient and remainder
    if quotient is None:
        quotient = np.zeros(len(dividend),dtype = int)

    remainder = np.zeros(len(dividend),dtype = int)

    #Case: If dividend is 0
    dividendZero = True 
    for i in range(0,length):
        if (dividend[i]!=0):
            dividendZero = False
            break
    if (dividendZero):
        return quotient,remainder

    #If degree of dividend is smaller than that of divisor
    if (deg_divisor > deg_dividend):
        remainder = dividend
        return quotient, remainder
    
    shift = deg_dividend - deg_divisor

    # Setting the remainder
    for i in range(0,length):
        if (i<shift):
            remainder[i] = dividend[i]
        else :
            remainder[i] = (dividend[i] + divisor[i-shift])%2 
    
    #Updating the quotient
    quotient[shift] = (quotient[shift] + 1)%2

    #Recursively calling field_div
    quotient, remainder = field_div(remainder,divisor,quotient) 
   
    return quotient,remainder 

In [6]:
def field_mul(a,b,p=2,n = 9,rigid = False, mod = False):
    '''
    Multiply two polynomial a and b in Z_p[x]
    Arguments: n: minimum returned size of final array
               a,b: Polynomials to be multiplied
               p : Field of coefficients
               rigid: Indicates if fn should stick to length n for array
               mod -> If set true, it will return the quotient ring group element 
                    of the final answer in Z_2[x]/ (x^8 + x^4 + x^3 + x + 1)
    '''
    ans = np.zeros(n, dtype = int)
    length_a = len(a)
    length_b = len(b)
        
    for i in range(0,n):
        if (b[i]==0):
            continue

        for k in range(i,n):
            ans[k] = int((ans[k] + b[i]*a[k-i])%p)
    
    z = [1,1,0,1,1,0,0,0,1]        # (x^8 + x^4 + x^3 + x + 1)
    if (mod):
        _,ans = field_div(ans,z)

    return ans  


In [7]:
def extended_euclid(x,y,n,p=2, details = False):
    '''
        Calculates a and b such that ax + by = 1  where a,b,x,y belong to Z_p[x]
    '''
    x = np.array(x,dtype=int)
    y = np.array(y, dtype= int)
    quotient, remainder = field_div(x,y)
    
    #If remainder is one
    rem_one = True
    for i in range(1,len(remainder)):
        if (remainder[i]!=0):
            rem_one = False
            break
    if rem_one and remainder[0]==1:
        return remainder, ((-1*quotient)%p)
    
    a,b = extended_euclid(y,remainder,n,p,details)

    k = b
    l = (a +  (-1*field_mul(b,quotient,p,n))%p )%p 

    #Prints log details about the calculation
    if details:
        print(print_poly(k) + print_poly(x) + ' + ' + print_poly(l) + print_poly(y) + ' = 1 \n')

    return k,l



 FIELDINV(poly): Calculate Inverse of polynomial 'poly' in the field $Z_{2}[x] / {(x^8 + x^4 + x^3 + x + 1)}$

In [8]:
def FIELD_INV(z,details=False):
    '''
    Calculated field inverse of polynomial z in Z_2[x]/(x^8 + x^4 + x^3 + x + 1)
    '''

    #CASE 1: z = 1  -> Return 1
    z_one = True
    if (z[0]!=1):
        z_one = False
    for i in range(1,9):
        if (z[i]!=0):
            z_one = False
    if (z_one==True):
        return z

    #General Case
    a = [1,1,0,1,1,0,0,0,1]     # (x^8 + x^4 + x^3 + x + 1)
    n = 9                       # Length of array
    m,n = extended_euclid(a,z,n,2,details)
    return n

In [9]:
def SUBBYTES(x, details = False):
    '''
    Performs the Sub-bytes transformation on hexadecimal value (XY)_16
    '''

    #Transforming (X,Y) hex --> [z0,z1,z2....z8] binary  
    z = np.zeros(9, dtype = int)
    x = int(x,16)
    for i in range(0,8):
        z[i] = x%2
        x=x//2

    #Checks if z is 0
    z_0 = True
    for i in range(0,8):
        if (z[i]!=0):
            z_0 = False
            break
    
    #Find inverse if not zero
    if z_0==False:
        z = FIELD_INV(z,details)

    C = [1,1,0,0,0,1,1,0]

    b = np.zeros(8,dtype = int)
    for i in range(0,8):
        b[i] = (z[i] + z[(i+4)%8] + z[(i+5)%8] + z[(i+6)%8]  + z[(i+7)%8] + C[i])%2

    #Converting back to hex-code
    m=0
    for i in range(0,8):
        if (b[i]==1):
            m = m + 2**i
    m = hex(m)

    return m

### S-BOX
We use the above SUBBYTE function to create an SBox table from which we can directly extract the SUBBYTE transformation
for an 8-bit number

In [10]:
# Uses SUBBYTE to build S_box dataframe
sbox = []
for i in range(0,16):
    sbox.append([])
    for j in range(0,16):
        code  = hex(i*16+j)
        sbox[i].append(SUBBYTES(code))

In [11]:
#Initializing index
ind = []
for i in range(0,16):
    ind.append(hex(i))

SBox= pd.DataFrame(sbox,columns = ind, index =ind)

In [12]:
SBox.head()

,0x0,0x1,0x2,0x3,0x4,0x5,0x6,0x7,0x8,0x9,0xa,0xb,0xc,0xd,0xe,0xf
0x0,0x63,0x7c,0x77,0x7b,0xf2,0x6b,0x6f,0xc5,0x30,0x1,0x67,0x2b,0xfe,0xd7,0xab,0x76
0x1,0xca,0x82,0xc9,0x7d,0xfa,0x59,0x47,0xf0,0xad,0xd4,0xa2,0xaf,0x9c,0xa4,0x72,0xc0
0x2,0xb7,0xfd,0x93,0x26,0x36,0x3f,0xf7,0xcc,0x34,0xa5,0xe5,0xf1,0x71,0xd8,0x31,0x15
0x3,0x4,0xc7,0x23,0xc3,0x18,0x96,0x5,0x9a,0x7,0x12,0x80,0xe2,0xeb,0x27,0xb2,0x75
0x4,0x9,0x83,0x2c,0x1a,0x1b,0x6e,0x5a,0xa0,0x52,0x3b,0xd6,0xb3,0x29,0xe3,0x2f,0x84


In [13]:
def SubBox(state):
    '''
    Method that takes in a state, performs SUBBYTE operation on each cell and returns the new state
    Argument: State
    Return: New State
    '''
    val  = 0
    for i in range(0,4):
        for j in range(0,4):
            val = int(state[i][j],16)
            X = val//16
            Y = val%16
            state[i][j] = SBox.iloc[X][Y]
    
    return state
    

### Shift Row Method 
$$
\begin{pmatrix}
S_{00} & S_{01} & S_{02} & S_{03}\\  
S_{10} & S_{11} & S_{12} & S_{13}\\ 
S_{20} & S_{21} & S_{22} & S_{33}\\ 
S_{30} & S_{31} & S_{32} & S_{33}
\end{pmatrix}
---ShiftRow--->
\begin{pmatrix}
S_{00} & S_{01} & S_{02} & S_{03}\\  
S_{11} & S_{12} & S_{13} & S_{10}\\ 
S_{22} & S_{23} & S_{20} & S_{31}\\ 
S_{33} & S_{30} & S_{31} & S_{32}
\end{pmatrix} 
$$

- Shift 0th row by 0
- Shift 1st row by 1 (left)
- Shift 2nd row by 2 (left)
- Shift 3rd row by 3 (left)

In [14]:
def ShiftRow(state):
    '''
        Performs the shift row operation
        Arguments -> state: the current state array
        Returns -> newState: state obtained after transformation
    '''
    newState = state.copy()
    for i in range (1,4):
        for j in range(0,4):
            newState[i][j] = state[i][(j+i)%4]
    return newState 

### Mix Column Transformation
$$
\begin{pmatrix}
02 && 03 && 01 && 01\\
01 && 02 && 03 && 01\\
01 && 01 && 02 && 03\\
03 && 01 && 01 && 02
\end{pmatrix}
.
\begin{pmatrix}
S_{00} & S_{01} & S_{02} & S_{03}\\  
S_{10} & S_{11} & S_{12} & S_{13}\\ 
S_{20} & S_{21} & S_{22} & S_{33}\\ 
S_{30} & S_{31} & S_{32} & S_{33}
\end{pmatrix}
=
\begin{pmatrix}
S'_{00} & S'_{01} & S'_{02} & S'_{03}\\  
S'_{11} & S'_{12} & S'_{13} & S'_{10}\\ 
S'_{22} & S'_{23} & S'_{20} & S'_{31}\\ 
S'_{33} & S'_{30} & S'_{31} & S'_{32}
\end{pmatrix} 
$$
where $S'_{ij} = \sum_{k=0}^{3} C_{ik}.S_{kj} $

(.) field multiplication  (Group multiplication operator)

($\oplus$) Bitwise XOR (Group addition operator)


In [15]:
def MixColumn(state):
    '''
    Implements the MixColumn transformation
    Argument: State
    Returns: newState
    '''
    
    for j in range(0,4):
        S = ['0x00']*4

        for i in range(0,4):
            S[i] = state[i][j]

        #Factors 
        factor = []
        a = convert_hex_to_bin_array('0x2')
        b = convert_hex_to_bin_array('0x3')
        c = convert_hex_to_bin_array('0x1')
        factor.append(a)
        factor.append(b)
        factor.append(c)
        factor.append(c)

        for i in range(0,4):
            temp = np.zeros(9,dtype=int)
            for k in range(0,4):
                bin_temp = convert_hex_to_bin_array(S[(i+k)%4])
                temp = temp ^ field_mul(bin_temp, factor[k],mod = True)       #Bitwise-XOR
            state[i][j] = convert_bin_array_to_hex(temp)
    
    return state
        

## Key Scheduling Algorithm

In [16]:
def RCON(i):
    # Defining the RCON array
    RCON = ['01','02','04','08','10','10','20','40','80','1B','36']
    RCON = [x+'000000' for x in RCON ]
    return RCON[i-1]

In [17]:
def ROTWARD(key):
    '''
        Implements the ROTWARD function on 4 byte data
    '''
    key = list(key)
    x= key[0:2]
    for i in range(2,8):
        key[i-2]=key[i]
    key[-2],key[-1]=x[0],x[1]
    return ''.join(key)

In [18]:
def SUBWORD(key):
    '''
        Implements the subbyte operation on 4 byte data
    '''

    newKey=''
    for i in range(0,4):
        val = int(key[i*2 : (i+1)*2],16)
        x = val//16
        y = val%16
        s = SBox.iloc[x][y][2:]
        if len(s)==1:
            s = '0'+s
        newKey = newKey + s
    
    return newKey

#SUBWORD('AB234F12')

In [19]:
def BitwiseWord(a,b):
    '''
        Performs bitwise operation on 2 strings representing hex values of a word each
    '''
    c = ''
    for i in range(0,8):
        c_i = convert_hex_to_bin_array(a[i])^convert_hex_to_bin_array(b[i])
        c = c + convert_bin_array_to_hex(c_i)[2:]
     
    return c

In [20]:
def KeyExpansion(key, n = 128, rounds = 12):
    '''
        Key Scheduling Algorithm that generates 12 keys of 16 bits each for AES encryption
        Arguments: key ->Secret Key (Default: 128 bit key)
                   n -> Length of secret key
                   rounds -> No. of keys to generate
        Returns:keyList -> List of generated keys
    '''

    keyWordLen = n//32           #no. of words in keys
    word = []

    #Initialize word
    for i in range(0,keyWordLen):
        word.append(key[8*i : 8*i + 8])
    
    totalWords = rounds*keyWordLen

    #Complete word
    for i in range(keyWordLen,totalWords):
        temp = word[i-1]
        if (i%keyWordLen == 0):
            temp = BitwiseWord( SUBWORD(ROTWARD(temp)), RCON(i//keyWordLen) )
        word.append(BitwiseWord(word[i-keyWordLen], temp))
    
    #Join Words to form keys
    keyList = []
    i=0
    while (i<totalWords):
        c = ''
        for j in range(0,keyWordLen):
            c+=word[i]
            i+=1
        keyList.append(c)

    return keyList

#KeyExpansion('ABC39F12ABCD5F12AF9DA612ABC89022', n=128)

In [21]:
# Some miscellaneous helper functions for AES encryption

def string_to_state(arr):
    '''
        Converts a string of 16 bytes to 4*4 bytes matrix
    '''
    state = []
    for i in range(0,4):
        state.append([])
        for j in range(0,4):
            state[i].append( arr[(i + 4*j)*2: (i + 4*j + 1)*2] )  
    return state

def state_to_string(s):
    '''
        Converts state back to string
    '''
    code = ''
    for j in range(0,4):
        for i in range(0,4):
            c = s[i][j][2:]
            if (len(c)==1):
                c = '0' + c
            code += c
    return code

#a = string_to_state('ABC39F12ABCD5F12AF9DA612ABC89022')
#b =  string_to_state('98C39F12ABCD5F36AF9DA612ABC89022')
#bitwiseState(a,b)

In [22]:
def AddRoundKey(s_a, key):
    '''
        Performs Add Round Key operation on state
    '''
    s_b = string_to_state(key)
    for i in range(0,4):
        for j in range(0,4):
            a = convert_hex_to_bin_array(s_a[i][j])
            b = convert_hex_to_bin_array(s_b[i][j])
            a = a^b 
            s_a[i][j] = convert_bin_array_to_hex(a)

    return s_a

In [23]:
def AES_encrypt(plainText, key, rounds = 10, blockSize = 128):
    '''
        Argument: plainText -> plainText that has to be encrypted using AES
                  key -> Secret key used for encrypting
                  rounds -> Number of rounds to be performed
        Returns:  code -> cipherText
    '''
    blockNibbleLen = blockSize//4
    X = plainText

    #Add padding of 0 incase plainText size is not divisible by blockSize 
    if (len(X)%blockNibbleLen != 0):
        padLen = blockNibbleLen - (len(X)%blockNibbleLen)
        pad = '0'*padLen
        X = X + pad

    X_list = []
    numOfBlocks = len(X)//blockNibbleLen
    for i in range(0,numOfBlocks):
        X_list.append(X[blockNibbleLen*i: blockNibbleLen*(i+1)])
    
    keyList = KeyExpansion(key, rounds = rounds+2, n = 128)

    cipherText = ''
    
    for x in X_list:
        state = string_to_state(x)

        k = keyList[0]
        state = AddRoundKey(state,k)
        
        # ( round-1 )  state transformation rounds
        for t in range(0,rounds-1):
            state = SubBox(state)
            state = ShiftRow(state)
            state = MixColumn(state)
            state = AddRoundKey(state, keyList[t+1])
        
        state = SubBox(state)
        state = ShiftRow(state)
        state = AddRoundKey(state, keyList[rounds])
        code = state_to_string(state)
        cipherText+= code

    return cipherText

#AES_encrypt('AB', '2', blockSize = 12)
    

Example: 

In [24]:
message = "My Name is Sombr3ro. Life isn't as amazing as it sounds, Padho Likho Zindagi jeeyo"
m = hexlify(message.encode()).decode()
print("Message is: \n" + message )
print("\nPlainText is "+ str(len(m)*4) +" bits long:\n" + m)

key = 'ABC39F12ABCD5F12AF9DA612ABC89022'

#Performing AES
cipherText = AES_encrypt(m,key)

print("\n\nCipherText is " + str(len(cipherText)*4) +" bits long:\n"+ cipherText)

Message is: 
My Name is Sombr3ro. Life isn't as amazing as it sounds, Padho Likho Zindagi jeeyo

PlainText is 656 bits long:
4d79204e616d6520697320536f6d627233726f2e204c6966652069736e277420617320616d617a696e6720617320697420736f756e64732c20506164686f204c696b686f205a696e64616769206a6565796f


CipherText is 768 bits long:
91ff74dde919faf79df61ce89d047b8c2a9efdd2470b35f83e6395e73065b4838371524096974d6afd9c3a75c58acc11cb737b43b5ba7969356c13768e88f8126f8e4dff9fbeb0d5aa8a25ca297531ae21ba16602818714a2e077e554a41f031


In [25]:
# To Test any transformations, use this
a = [['0x63','0xeb','0x9f','0xa0'],['0x2f','0x93','0x92','0xC0'],['0xaf','0xc7','0xab','0x30'],['0xa2','0x20','0xcb','0x2b']]
MixColumn(a)

[['0xba', '0x84', '0xe8', '0x1b'],
 ['0x75', '0xa4', '0x8d', '0x40'],
 ['0xf4', '0x8d', '0x6', '0x7d'],
 ['0x7a', '0x32', '0xe', '0x5d']]